In [1]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import GridSearchCV, cross_val_score
#from sklearn.decomposition import PCA
#from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBooster
#from sklearn.preprocessing import StandardScaler,MinMaxScaler
#from sklearn.svm import SVR
#from fancyimpute import MICE
from surprise import SVD, NMF, KNNBasic
from surprise import dataset
from surprise.model_selection import cross_validate

In [4]:
train_path = "C:\\Users\\Pavani Gangula\\Downloads\\train_black.zip"
test_path = "C:\\Users\\Pavani Gangula\\Downloads\\test_Black.zip"

In [5]:
train, test = pd.read_csv(train_path), pd.read_csv(test_path)

In [6]:
train.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871


In [7]:
test.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
5,1000013,P00350442,M,46-50,1,C,3,1,2,3.0,15.0
6,1000013,P00155442,M,46-50,1,C,3,1,1,11.0,15.0
7,1000013,P0094542,M,46-50,1,C,3,1,2,4.0,9.0
8,1000015,P00161842,M,26-35,7,A,1,0,10,13.0,16.0
9,1000022,P00067942,M,18-25,15,A,4+,0,5,14.0,NaN


In [8]:
train.shape

(550068, 12)

In [9]:
test.shape

(233599, 11)

In [10]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [11]:
test.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
dtype: object

In [12]:
train['Purchase_in_th'] = train['Purchase']/1000

In [13]:
train['User_ID'].value_counts()

1001680    1026
1004277     979
1001941     898
1001181     862
1000889     823
1003618     767
1001150     752
1001015     740
1005795     729
1005831     727
1002909     718
1001449     714
1002063     709
1004344     705
1003391     698
1003841     698
1000424     694
1004510     691
1001980     685
1001088     680
1004227     676
1003808     671
1004508     651
1000549     632
1003224     622
1003539     617
1005367     612
1001285     606
1004543     588
1003824     584
           ... 
1001318       9
1004178       9
1004464       9
1005043       9
1003388       9
1003883       9
1005117       9
1002160       9
1002893       9
1002061       9
1004068       9
1002037       9
1004527       8
1003291       8
1002965       8
1002488       8
1000320       8
1003542       8
1002135       8
1003275       8
1005904       8
1004192       8
1005810       7
1005391       7
1002690       7
1000094       7
1002111       7
1004991       7
1005608       7
1000708       6
Name: User_ID, Length: 5

In [14]:
set(train.User_ID)-set(test.User_ID)

set()

In [15]:
len(set(train.Product_ID)-set(test.Product_ID))

186

In [16]:
len(set(test.Product_ID)-set(train.Product_ID))

46

In [17]:
train.replace(np.nan,99,inplace=True)

In [18]:
train.Product_Category_1 = train.Product_Category_1.astype('category')
train.Product_Category_2 = train.Product_Category_2.astype('category')
train.Product_Category_3 = train.Product_Category_3.astype('category')

In [19]:
print(len(set(train.Product_ID)))
print(len(set(train.User_ID)))
print(len(set(train.Product_Category_1)))
print(len(set(train.Product_Category_2)))
print(len(set(train.Product_Category_3)))

3631
5891
20
18
16


In [20]:
from surprise.reader import Reader

In [21]:
r = Reader(rating_scale=(0,24))

In [22]:
D = dataset.Dataset.load_from_df(train[['User_ID','Product_ID','Purchase_in_th']],reader = r)

In [23]:
cross_validate(SVD(),D,cv=5)

{'fit_time': (57.477861642837524,
  57.4368679523468,
  57.431517601013184,
  57.615230083465576,
  57.55318355560303),
 'test_mae': array([1.96884203, 1.96319565, 1.96815258, 1.9677118 , 1.96710219]),
 'test_rmse': array([2.69044669, 2.68702286, 2.68993599, 2.69892343, 2.69361121]),
 'test_time': (2.5729572772979736,
  1.7953157424926758,
  2.022874593734741,
  1.8085432052612305,
  1.9659852981567383)}

In [24]:
NMF()

In [25]:
np.array(D)

array(<surprise.dataset.DatasetAutoFolds object at 0x0000023D2B6DD128>,
      dtype=object)

In [26]:
S = SVD(n_factors = 500)

In [27]:
d1 = D.build_full_trainset()

In [28]:
S.fit(d1)

In [29]:
#S.predict(uid='1000004',iid='P00128942')[3]

In [30]:
pred_test = []
for u,i in zip(test.User_ID,test.Product_ID):
    pred = S.predict(uid=u,iid=i)[3]
    pred = pred*1000
    pred_test.append((u,i,pred))

In [31]:
pred1 = pd.DataFrame(pred_test,columns=['User_ID','Product_ID','Purchase'])
pred1.to_csv("C:\\Users\\Pavani Gangula\\Desktop\\Submission.csv")